### Importação de arquivo

In [2]:
import sys
file_loc = '/Users/thalesvieira/Projects/sibgrapi2022/data'
sys.path.insert(0,'/store/karelia/sibgrapi2023')

import CityHub
import networkx as nx
import numpy as np
import osmnx as ox
#cg=CityHub.load_preprocessed_CityHub('/store/karelia/sp3.bin')#(file_loc+'/sp-ordena.bin')
#G=cg.city_mesh_graph#city_street_graph

In [ ]:
## Para servidor
# import sys
# sys.path.insert(0,'/store/karelia/')
# from CityGraph import *
# import CityGraph
# cg=CityGraph.load_preprocessed_CityGraph('/store/tvieira/CityGraph/shared/data/sp-completo.bin')

### Download de cidade via string

In [ ]:
cg = CityHub.CityHub('Maceió, Brazil',40.0)
G=cg.city_street_graph

### Funções auxiliares

In [54]:
# https://mail.python.org/pipermail/scipy-dev/2014-October/020120.html
from scipy.sparse import coo_matrix
def is_symmetric(m):
  if m.shape[0] != m.shape[1]:
    raise ValueError('m must be a square matrix')
  if not isinstance(m, coo_matrix):
    m = coo_matrix(m)
  r, c, v = m.row, m.col, m.data
  tril_no_diag = r > c
  triu_no_diag = c > r

  if triu_no_diag.sum() != tril_no_diag.sum():
    return False
  rl = r[tril_no_diag]
  cl = c[tril_no_diag]
  vl = v[tril_no_diag]
  ru = r[triu_no_diag]
  cu = c[triu_no_diag]
  vu = v[triu_no_diag]

  sortl = np.lexsort((cl, rl))
  sortu = np.lexsort((ru, cu))
  vl = vl[sortl]
  vu = vu[sortu]

  check = np.allclose(vl, vu)

  return check

def is_triangular(A):
  m = coo_matrix(A)
  r, c, v = m.row, m.col, m.data
  tril_no_diag = r > c
  triu_no_diag = c > r
  if triu_no_diag.sum()==0 or tril_no_diag.sum()==0 :
    return True
  return False

## Laplacian e Fiedler

In [55]:
# The graph is connected?
print(nx.is_connected(G))
print(nx.number_connected_components(G))

True
1


In [56]:
import osmnx as ox
import math

'''' If you want to use the average distance between all point pairs as sigma (slow)
soma=0.0
for u in G.nodes:
    for v in G.nodes:
        if u!=v:
            vertA = cg.city_vert_list[cg.city_vert_nxind_to_ind_dict[u]]
            vertB = cg.city_vert_list[cg.city_vert_nxind_to_ind_dict[v]]            
            soma+= ox.distance.great_circle_vec(vertA[0],vertA[1],vertB[0],vertB[1])
sigma=soma/(len(G.nodes)*(len(G.nodes)-1))
'''

sigma=5000

# Inverse length
for u in G.edges:
   G[u[0]][u[1]][0]["inv_length"] = 1/G[u[0]][u[1]][0]["length"] 
# Gaussian:
#    G[u[0]][u[1]][0]["inv_length"] = math.exp(-(G[u[0]][u[1]][0]["length"]**2)/(sigma**2)) 
    
laplacian = nx.laplacian_matrix(G,weight='inv_length')

# Is Symmetric?
print(is_symmetric(laplacian))
# Is Triangular?
print(is_triangular(laplacian))

True
False


In [57]:
from scipy.sparse.linalg import eigs
eig_values, eig_vectors = eigs(laplacian.astype(float),k=2, which='LM', sigma=0)
fiedler_pos = np.where(eig_values.real == np.sort(eig_values.real)[1])[0][0]
fiedler_vector = np.transpose(eig_vectors)[fiedler_pos]

print("Fiedler vector: " + str(fiedler_vector.real))
fiedler_vector_real = fiedler_vector.real
fiedler_sorted_index_list = np.argsort(np.argsort(fiedler_vector_real))

i=0
for u in G.nodes:
    G.nodes[u]['fiedler_index']=fiedler_sorted_index_list[i]
    i=i+1
    
inverse_fiedler_sorted_index_list = [0] * len(cg.city_vert_list)
for i in range(len(fiedler_sorted_index_list)):
    inverse_fiedler_sorted_index_list[fiedler_sorted_index_list[i]]=i

Fiedler vector: [-3.88640009e-03 -4.07912538e-03 -4.68661359e-03 ... -2.55740329e-03
 -2.55746591e-03  6.37940032e-05]


## Random

In [58]:
random_index_list = np.random.permutation(len(cg.city_vert_list))

## t-SNE 1D

In [59]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=1, verbose=1, random_state=123)
tsne_embedding = tsne.fit_transform(np.array(cg.city_vert_list)) 

tsne_sorted_index_list = np.argsort(np.argsort(tsne_embedding.flatten()))

inverse_tsne_sorted_index_list = [0] * len(cg.city_vert_list)
for i in range(len(tsne_sorted_index_list)):
    inverse_tsne_sorted_index_list[tsne_sorted_index_list[i]]=i

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 16088 samples in 0.009s...
[t-SNE] Computed neighbors for 16088 samples in 0.235s...
[t-SNE] Computed conditional probabilities for sample 1000 / 16088
[t-SNE] Computed conditional probabilities for sample 2000 / 16088
[t-SNE] Computed conditional probabilities for sample 3000 / 16088
[t-SNE] Computed conditional probabilities for sample 4000 / 16088
[t-SNE] Computed conditional probabilities for sample 5000 / 16088
[t-SNE] Computed conditional probabilities for sample 6000 / 16088
[t-SNE] Computed conditional probabilities for sample 7000 / 16088
[t-SNE] Computed conditional probabilities for sample 8000 / 16088
[t-SNE] Computed conditional probabilities for sample 9000 / 16088
[t-SNE] Computed conditional probabilities for sample 10000 / 16088
[t-SNE] Computed conditional probabilities for sample 11000 / 16088
[t-SNE] Computed conditional probabilities for sample 12000 / 16088
[t-SNE] Computed conditional probabilities for sam

## Métrica de qualidade sentido embedding -> latlong

### original

In [61]:
from sklearn.metrics.pairwise import pairwise_distances,haversine_distances

window_size = int(len(cg.city_vert_list)/100)

diagonals_original = []

for i in range(0,len(cg.city_vert_list)-window_size,1):
    Lats = [cg.city_vert_list[j][0] for j in range(i,i+window_size)]
    Longs = [cg.city_vert_list[j][1] for j in range(i,i+window_size)]
    min_rad = [math.radians(_) for _ in [min(Lats),min(Longs)]]
    max_rad = [math.radians(_) for _ in [max(Lats),max(Longs)]]
    
    diagonals_original.append(haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000)

TypeError: 'numpy.int64' object is not callable

### Fiedler

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances,haversine_distances
import math


diagonals_fiedler = []

for i in range(0,len(cg.city_vert_list)-window_size,1):
    Lats = [cg.city_vert_list[j][0] for j in inverse_fiedler_sorted_index_list[i:i+window_size]]
    Longs = [cg.city_vert_list[j][1] for j in inverse_fiedler_sorted_index_list[i:i+window_size]]
    min_rad = [math.radians(_) for _ in [min(Lats),min(Longs)]]
    max_rad = [math.radians(_) for _ in [max(Lats),max(Longs)]]
    
    diagonals_fiedler.append(haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000)

### t-SNE

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances,haversine_distances
import math


diagonals_tsne = []

for i in range(0,len(cg.city_vert_list)-window_size,1):
    Lats = [cg.city_vert_list[j][0] for j in inverse_tsne_sorted_index_list[i:i+window_size]]
    Longs = [cg.city_vert_list[j][1] for j in inverse_tsne_sorted_index_list[i:i+window_size]]
    min_rad = [math.radians(_) for _ in [min(Lats),min(Longs)]]
    max_rad = [math.radians(_) for _ in [max(Lats),max(Longs)]]
    
    diagonals_tsne.append(haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000)

### Random

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances,haversine_distances

diagonals_random = []

for i in range(0,len(cg.city_vert_list)-window_size,1):
#for i in range(0,10000,1000):
    Lats = [cg.city_vert_list[j][0] for j in random_index_list[i:i+window_size]]
    Longs = [cg.city_vert_list[j][1] for j in random_index_list[i:i+window_size]]
    min_rad = [math.radians(_) for _ in [np.min(Lats),np.min(Longs)]]
    max_rad = [math.radians(_) for _ in [np.max(Lats),np.max(Longs)]]
    
    diagonals_random.append(haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000)
   # print(diagonals_random[-1],' ',i/100,' de ',(len(cg.city_vert_list)-window_size)/100)
    
#    break

In [ ]:
import statistics
print('Fiedler - média:',statistics.mean(diagonals_fiedler))
print('Fiedler - mediana:',statistics.median(diagonals_fiedler))
print('t-SNE - média:',statistics.mean(diagonals_tsne))
print('t-SNE - mediana:',statistics.median(diagonals_tsne))
print('Original - média:',statistics.mean(diagonals_original))
print('Original - mediana:',statistics.median(diagonals_original))
print('Random - média:',statistics.mean(diagonals_random))
print('Random - mediana:',statistics.median(diagonals_random))

In [ ]:
import matplotlib.pyplot as plt
plt.hist([diagonals_fiedler, diagonals_tsne,diagonals_original, diagonals_random], bins=40, label=['fiedler', 't-sne', 'original', 'random'], color=['blue','orange','red', 'green'],range=[0,40])
plt.legend(loc='upper right')
plt.show()

## Métrica de qualidade sentido latlong -> embedding

### Fiedler, original and random

In [ ]:
max_inverse_ind_dist_fiedler = []
max_inverse_ind_dist_original = []
max_inverse_ind_dist_random = []
max_inverse_ind_dist_tsne = []

for i in range(0,len(cg.city_vert_list)):
    ps=nx.single_source_dijkstra_path(cg.city_street_graph,cg.city_vert_ind_to_nxind_dict[i],cutoff=0.5*1000.0,weight='length')
    result_nodes = []
    for k in ps.keys():
        result_nodes.append(cg.city_vert_nxind_to_ind_dict[k])  
    inds_sorted_fiedler = [fiedler_sorted_index_list[j] for j in result_nodes]
    inds_sorted_tsne = [tsne_sorted_index_list[j] for j in result_nodes]
    inds_random = [random_index_list[j] for j in result_nodes]
    max_inverse_ind_dist_fiedler.append((np.max(inds_sorted_fiedler)-np.min(inds_sorted_fiedler))/len(inds_sorted_fiedler))  
    max_inverse_ind_dist_tsne.append((np.max(inds_sorted_tsne)-np.min(inds_sorted_tsne))/len(inds_sorted_tsne))  
    max_inverse_ind_dist_original.append((np.max(result_nodes)-np.min(result_nodes))/len(result_nodes))
    max_inverse_ind_dist_random.append((np.max(inds_random)-np.min(inds_random))/len(inds_random))  

In [ ]:
plt.hist([max_inverse_ind_dist_fiedler,max_inverse_ind_dist_fiedler,max_inverse_ind_dist_original,max_inverse_ind_dist_random], bins=40, label=['fiedler','t-sne','original','random'], color=['blue', 'orange','red','green'],range=[0,400])
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.hist([max_inverse_ind_dist_fiedler,max_inverse_ind_dist_tsne,max_inverse_ind_dist_original,max_inverse_ind_dist_random], bins=40, label=['fiedler','t-sne','original','random'], color=['blue', 'orange','red','green'],range=[0,50])
plt.legend(loc='upper right')
plt.show()

In [ ]:
import statistics
print('Fiedler - média:',statistics.mean(max_inverse_ind_dist_fiedler))
print('Fiedler - mediana:',statistics.median(max_inverse_ind_dist_fiedler))
print('t-SNE - média:',statistics.mean(max_inverse_ind_dist_tsne))
print('t-SNE - mediana:',statistics.median(max_inverse_ind_dist_tsne))
print('Original - média:',statistics.mean(max_inverse_ind_dist_original))
print('Original - mediana:',statistics.median(max_inverse_ind_dist_original))
print('Random - média:',statistics.mean(max_inverse_ind_dist_random))
print('Random - mediana:',statistics.median(max_inverse_ind_dist_random))

## Mapa com colormap pela ordenação do vetor de Fiedler

In [ ]:
import folium
import branca
import branca.colormap as cm

#colormap = cm.LinearColormap(colors=['blue','yellow'],vmin=np.amin(sorted_index_list) ,vmax=np.amax(sorted_index_list))
min= np.amin(fiedler_sorted_index_list)
max = np.amax(fiedler_sorted_index_list)
colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
                             index=[min, max/5, max/5*2, max/5*3, max/5*4, max], vmin=min, vmax=max,
                             caption='Fiedler index')

Lats = [cg.city_vert_list[j][0] for j in range(len(cg.city_vert_list))]
Longs = [cg.city_vert_list[j][1] for j in range(len(cg.city_vert_list))]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list)):
     folium.CircleMarker(cg.city_vert_list[inverse_fiedler_sorted_index_list[esquina]], radius=1, color=colormap(esquina),opacity=1).add_to(m)  

   
m.add_child(colormap)
m
m.save('fiedler_map_fiedler.html')

## Mapa com colormap pela ordenação do t-SNE

In [ ]:
import folium
import branca
import branca.colormap as cm

#colormap = cm.LinearColormap(colors=['blue','yellow'],vmin=np.amin(sorted_index_list) ,vmax=np.amax(sorted_index_list))
min= np.amin(tsne_sorted_index_list)
max = np.amax(tsne_sorted_index_list)
colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
                             index=[min, max/5, max/5*2, max/5*3, max/5*4, max], vmin=min, vmax=max,
                             caption='Fiedler index')

Lats = [cg.city_vert_list[j][0] for j in range(len(cg.city_vert_list))]
Longs = [cg.city_vert_list[j][1] for j in range(len(cg.city_vert_list))]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list)):
     folium.CircleMarker(cg.city_vert_list[inverse_tsne_sorted_index_list[esquina]], radius=1, color=colormap(esquina),opacity=1).add_to(m)  

   
m.add_child(colormap)
m
m.save('fiedler_map_tsne.html')

## Mapa com colormap pela métrica de qualidade inversa do vetor de Fiedler

In [ ]:
import folium
import branca
import branca.colormap as cm

#colormap = cm.LinearColormap(colors=['blue','yellow'],vmin=np.amin(sorted_index_list) ,vmax=np.amax(sorted_index_list))
min= np.amin(0)
max = np.max(max_inverse_ind_dist_fiedler)/16
colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
                             index=[min, max/5, max/5*2, max/5*3, max/5*4, max], vmin=min, vmax=max,
                             caption='inverse indices distance through fiedler error')

Lats = [cg.city_vert_list[j][0] for j in range(len(cg.city_vert_list))]
Longs = [cg.city_vert_list[j][1] for j in range(len(cg.city_vert_list))]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list)):
    folium.CircleMarker(cg.city_vert_list[esquina], radius=1, color=colormap(max_inverse_ind_dist_fiedler[esquina])).add_to(m)  
   
m.add_child(colormap)
m
m.save('fiedler_valor_map.html')